Sacar mi historial de partidas SUMMONER-V4, MATCH-V5
o 
info de los jugadores de una división concreta LEAGUE-V4 division

In [42]:
import requests
import pandas as pd
from creds import api_key
import re

In [2]:
url_summoner = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Masgt7'

Nos centramos en SUMMONER-V4que tiene que ver con cualquier cosa que haga una cuenta en concreto

In [3]:
url_summoner = url_summoner + '?api_key=' + api_key

In [4]:
r = requests.get (url_summoner)
r.status_code

200

In [5]:
player_info = r.json()
print (player_info)

{'id': '7xKp5xDyqk1u0_RKrh8SdaSHYhjpThiq7XXJscT2DQx273I', 'accountId': 'NcGfPSobpg1gNOsZBjrQNCVyeIg7MyijVoglw0WBQioOST0', 'puuid': 'uFIekhnHnZjaIXG_IG5y2Kb_UbBsEv57LksBUe6qMh0h0MPBKg8xoWtsbQkvXcPhlHLNGn8xZwqzGQ', 'name': 'Masgt7', 'profileIconId': 940, 'revisionDate': 1700743850091, 'summonerLevel': 74}


Obtenida la info de mi cuenta, ahora hacemos otra request para obtener identificadores de mis últimas partidas a partir del 'puuid'

In [6]:
puuid = player_info['puuid']

In [7]:
url_match_history = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?type=normal&start=0&count=20&api_key=' + api_key #Limitamos la lista de partidas a 20
r = requests.get(url_match_history)

In [8]:
if r.status_code == 200:
    game_id = r.json()
    print (game_id)
else:
    print ('Error', r.status_code, ':',)  

['EUW1_6688283685', 'EUW1_6688256964', 'EUW1_6688236897', 'EUW1_6688218767', 'EUW1_6686773929', 'EUW1_6686744190', 'EUW1_6684157765', 'EUW1_6684133215', 'EUW1_6682813114', 'EUW1_6682767095', 'EUW1_6682519608', 'EUW1_6678781224', 'EUW1_6678748896', 'EUW1_6678696426', 'EUW1_6678677984', 'EUW1_6678647009', 'EUW1_6677476957', 'EUW1_6677447766', 'EUW1_6673723132', 'EUW1_6673680411']


Vamos a sacar ahora información de cada partida y almacenarla en ficheros .csv por separado

In [9]:
counter = 1
for i in game_id:
    url_game_info = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + api_key
    game_info = requests.get(url_game_info)
    index_player = game_info.json()['metadata']['participants'].index(puuid)
    game_data = game_info.json()['info']['participants'][index_player]
    game_df = pd.json_normalize(game_data)
    game_df.to_csv ('games\\game' + str(counter) + '.csv', sep = ',', mode = 'x')
    counter += 1

Cargamos una lista con DF a partir de los CSV

In [13]:
list_df = []

for i in range(len(game_id)):
    list_df.append((pd.read_csv('games\\game' + str (i+1) + '.csv')))
    print (list_df[i].shape)

(1, 262)
(1, 264)
(1, 264)
(1, 264)
(1, 264)
(1, 263)
(1, 268)
(1, 269)
(1, 268)
(1, 269)
(1, 269)
(1, 268)
(1, 268)
(1, 267)
(1, 267)
(1, 267)
(1, 267)
(1, 266)
(1, 268)
(1, 268)


No tienen las mismas columnas, de modo que vamos a quedarnos únicamente con las que tengan en común

In [49]:
common_columns = list(set.intersection(*(set(df.columns) for df in list_df)))
common_columns[0:10] #Vemos algunas

['challenges.snowballsHit',
 'challenges.turretTakedowns',
 'challenges.quickSoloKills',
 'challenges.survivedSingleDigitHpCount',
 'challenges.soloBaronKills',
 'killingSprees',
 'pushPings',
 'challenges.wardTakedowns',
 'allInPings',
 'challenges.killParticipation']

In [116]:
df_joined = pd.concat(list_df)
df_joined = df_joined [common_columns]

df_joined.head()

,challenges.snowballsHit,challenges.turretTakedowns,challenges.quickSoloKills,challenges.survivedSingleDigitHpCount,challenges.soloBaronKills,killingSprees,pushPings,challenges.wardTakedowns,allInPings,challenges.killParticipation,inhibitorTakedowns,missions.playerScore3,magicDamageDealtToChampions,challenges.stealthWardsPlaced,challenges.multikillsAfterAggressiveFlash,challenges.initialCrabCount,pentaKills,physicalDamageDealtToChampions,challenges.jungleCsBefore10Minutes,missions.playerScore8,item1,objectivesStolen,spell2Casts,teamEarlySurrendered,challenges.bountyGold,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.laneMinionsFirst10Minutes,challenges.twentyMinionsIn3SecondsCount,baronKills,missions.playerScore6,wardsPlaced,challenges.teamBaronKills,enemyMissingPings,challenges.landSkillShotsEarlyGame,challenges.lostAnInhibitor,challenges.immobilizeAndKillWithAlly,challenges.elderDragonMultikills,spell3Casts,summoner2Id,challenges.firstTurretKilled,perks.statPerks.defense,kills,magicDamageTaken,challenges.abilityUses,challenges.completeSupportQuestInTime,missions.playerScore1,doubleKills,firstTowerAssist,riotIdTagline,challenges.epicMonsterKillsWithin30SecondsOfSpawn,missions.playerScore4,challenges.takedownOnFirstTurret,playerScore8,totalDamageDealtToChampions,playerScore9,item0,totalHealsOnTeammates,turretKills,bountyLevel,nexusKills,totalMinionsKilled,turretTakedowns,playerScore2,challenges.damagePerMinute,challenges.effectiveHealAndShielding,challenges.skillshotsDodged,visionClearedPings,challenges.mejaisFullStackInTime,playerAugment1,challenges.damageTakenOnTeamPercentage,summonerId,playerScore1,playerAugment4,totalUnitsHealed,challenges.enemyJungleMonsterKills,challenges.wardsGuarded,firstBloodAssist,playerSubteamId,challenges.visionScorePerMinute,missions.playerScore7,item5,totalDamageShieldedOnTeammates,challenges.survivedThreeImmobilizesInFight,inhibitorKills,playerScore11,damageDealtToTurrets,tripleKills,challenges.killsWithHelpFromEpicMonster,damageDealtToBuildings,challenges.dodgeSkillShotsSmallWindow,summoner1Id,needVisionPings,challenges.goldPerMinute,challenges.riftHeraldTakedowns,challenges.teamElderDragonKills,win,challenges.takedownsFirstXMinutes,challenges.doubleAces,playerScore5,challenges.scuttleCrabKills,puuid,challenges.multiKillOneSpell,assistMePings,holdPings,summoner1Casts,baitPings,objectivesStolenAssists,playerScore6,totalAllyJungleMinionsKilled,challenges.soloKills,challenges.outnumberedKills,visionScore,item2,perks.statPerks.flex,challenges.hadOpenNexus,perks.statPerks.offense,largestCriticalStrike,challenges.turretPlatesTaken,missions.playerScore0,summonerLevel,summoner2Casts,challenges.flawlessAces,playerScore3,challenges.killsUnderOwnTurret,Unnamed: 0,totalDamageDealt,physicalDamageTaken,challenges.enemyChampionImmobilizations,challenges.multikills,challenges.killsOnOtherLanesEarlyJungleAsLaner,subteamPlacement,challenges.dancedWithRiftHerald,physicalDamageDealt,perks.styles,getBackPings,gameEndedInSurrender,spell4Casts,inhibitorsLost,challenges.twoWardsOneSweeperCount,missions.playerScore9,detectorWardsPlaced,firstTowerKill,visionWardsBoughtInGame,challenges.acesBefore15Minutes,challenges.skillshotsHit,championTransform,totalEnemyJungleMinionsKilled,missions.playerScore11,challenges.fullTeamTakedown,missions.playerScore2,challenges.wardTakedownsBefore20M,damageDealtToObjectives,spell1Casts,challenges.killingSprees,challenges.legendaryCount,challenges.knockEnemyIntoTeamAndKill,challenges.junglerTakedownsNearDamagedEpicMonster,consumablesPurchased,wardsKilled,challenges.multiTurretRiftHeraldCount,challenges.outerTurretExecutesBefore10Minutes,challenges.gameLength,playerAugment3,lane,item4,challenges.epicMonsterSteals,assists,magicDamageDealt,challenges.saveAllyFromDeath,gameEndedInEarlySurrender,challenges.epicMonsterStolenWithoutSmite,challenges.pickKillWithAlly,challenges.unseenRecalls,trueDamageTaken,challenges.poroExplosions,challenges.killsNearEnemyTurret,commandPings,chall

Eliminamos las palabras previas al punto

In [ ]:
def remove_dot (string):
    pattern = r'.*?\.(.*)'
    result = re.match(pattern, string)
    if result:
        return result.group(1)
    else:
        return string

In [109]:
df_columns = list(df_joined.columns)

for i,j in enumerate(df_columns):
    df_columns[i] = remove_dot(j)

df_joined.columns = df_columns

df_joined.head(1)

,snowballsHit,turretTakedowns,quickSoloKills,survivedSingleDigitHpCount,soloBaronKills,killingSprees,pushPings,wardTakedowns,allInPings,killParticipation,inhibitorTakedowns,playerScore3,magicDamageDealtToChampions,stealthWardsPlaced,multikillsAfterAggressiveFlash,initialCrabCount,pentaKills,physicalDamageDealtToChampions,jungleCsBefore10Minutes,playerScore8,item1,objectivesStolen,spell2Casts,teamEarlySurrendered,bountyGold,getTakedownsInAllLanesEarlyJungleAsLaner,laneMinionsFirst10Minutes,twentyMinionsIn3SecondsCount,baronKills,playerScore6,wardsPlaced,teamBaronKills,enemyMissingPings,landSkillShotsEarlyGame,lostAnInhibitor,immobilizeAndKillWithAlly,elderDragonMultikills,spell3Casts,summoner2Id,firstTurretKilled,statPerks.defense,kills,magicDamageTaken,abilityUses,completeSupportQuestInTime,playerScore1,doubleKills,firstTowerAssist,riotIdTagline,epicMonsterKillsWithin30SecondsOfSpawn,playerScore4,takedownOnFirstTurret,playerScore8,totalDamageDealtToChampions,playerScore9,item0,totalHealsOnTeammates,turretKills,bountyLevel,nexusKills,totalMinionsKilled,turretTakedowns,playerScore2,damagePerMinute,effectiveHealAndShielding,skillshotsDodged,visionClearedPings,mejaisFullStackInTime,playerAugment1,damageTakenOnTeamPercentage,summonerId,playerScore1,playerAugment4,totalUnitsHealed,enemyJungleMonsterKills,wardsGuarded,firstBloodAssist,playerSubteamId,visionScorePerMinute,playerScore7,item5,totalDamageShieldedOnTeammates,survivedThreeImmobilizesInFight,inhibitorKills,playerScore11,damageDealtToTurrets,tripleKills,killsWithHelpFromEpicMonster,damageDealtToBuildings,dodgeSkillShotsSmallWindow,summoner1Id,needVisionPings,goldPerMinute,riftHeraldTakedowns,teamElderDragonKills,win,takedownsFirstXMinutes,doubleAces,playerScore5,scuttleCrabKills,puuid,multiKillOneSpell,assistMePings,holdPings,summoner1Casts,baitPings,objectivesStolenAssists,playerScore6,totalAllyJungleMinionsKilled,soloKills,outnumberedKills,visionScore,item2,statPerks.flex,hadOpenNexus,statPerks.offense,largestCriticalStrike,turretPlatesTaken,playerScore0,summonerLevel,summoner2Casts,flawlessAces,playerScore3,killsUnderOwnTurret,Unnamed: 0,totalDamageDealt,physicalDamageTaken,enemyChampionImmobilizations,multikills,killsOnOtherLanesEarlyJungleAsLaner,subteamPlacement,dancedWithRiftHerald,physicalDamageDealt,styles,getBackPings,gameEndedInSurrender,spell4Casts,inhibitorsLost,twoWardsOneSweeperCount,playerScore9,detectorWardsPlaced,firstTowerKill,visionWardsBoughtInGame,acesBefore15Minutes,skillshotsHit,championTransform,totalEnemyJungleMinionsKilled,playerScore11,fullTeamTakedown,playerScore2,wardTakedownsBefore20M,damageDealtToObjectives,spell1Casts,killingSprees,legendaryCount,knockEnemyIntoTeamAndKill,junglerTakedownsNearDamagedEpicMonster,consumablesPurchased,wardsKilled,multiTurretRiftHeraldCount,outerTurretExecutesBefore10Minutes,gameLength,playerAugment3,lane,item4,epicMonsterSteals,assists,magicDamageDealt,saveAllyFromDeath,gameEndedInEarlySurrender,epicMonsterStolenWithoutSmite,pickKillWithAlly,unseenRecalls,trueDamageTaken,poroExplosions,killsNearEnemyTurret,commandPings,12AssistStreakCount,controlWardsPlaced,participantId,placement,teamId,teamPosition,goldEarned,quickCleanse,basicPings,kTurretsDestroyedBeforePlatesFall,takedownsAfterGainingLevelAdvantage,initialBuffCount,teamRiftHeraldKills,baronTakedowns,kda,turretsTakenWithRiftHerald,championName,role,championId,goldSpent,individualPosition,epicMonsterKillsNearEnemyJungler,turretsLost,enemyVisionPings,itemsPurchased,outnumberedNexusKill,perfectGame,quickFirstTurret,onMyWayPings,item6,takedownsBeforeJungleMinionSpawn,totalTimeSpentDead,profileIcon,deaths,firstBloodKill,deathsByEnemyChamps,nexusLost,playerScore7,playerScore5,buffsStolen,damageSelfMitigated,timePlayed,perfectDragonSoulsTaken,neutralMinionsKilled,quadraKills,trueDamageDealtToChampions,alliedJungleMonsterKills,summonerName,takedownsInEnemyFountain,playerScore10,dragonTakedowns,elderDragonKillsWithOpposingSoul,playerAugment2,playerScore0,item3,totalHe

In [117]:
df_joined.shape

(20, 260)

We have a very high dimensionality in our dataset, so now we'll check the descriptive statistics of each variable, and we'll drop the columns that we consider not relevant

In [77]:
# display(common_rows)
pd.set_option('display.max_columns', None) #Quitamos el límite de columnas para poder ver todas y evaluar cuáles son importantes
display(df_joined.describe())

,snowballsHit,turretTakedowns,quickSoloKills,survivedSingleDigitHpCount,soloBaronKills,killingSprees,pushPings,wardTakedowns,allInPings,killParticipation,inhibitorTakedowns,playerScore3,magicDamageDealtToChampions,stealthWardsPlaced,multikillsAfterAggressiveFlash,initialCrabCount,pentaKills,physicalDamageDealtToChampions,jungleCsBefore10Minutes,playerScore8,item1,objectivesStolen,spell2Casts,bountyGold,getTakedownsInAllLanesEarlyJungleAsLaner,laneMinionsFirst10Minutes,twentyMinionsIn3SecondsCount,baronKills,playerScore6,wardsPlaced,teamBaronKills,enemyMissingPings,landSkillShotsEarlyGame,lostAnInhibitor,immobilizeAndKillWithAlly,elderDragonMultikills,spell3Casts,summoner2Id,firstTurretKilled,statPerks.defense,kills,magicDamageTaken,abilityUses,completeSupportQuestInTime,playerScore1,doubleKills,epicMonsterKillsWithin30SecondsOfSpawn,playerScore4,takedownOnFirstTurret,playerScore8,totalDamageDealtToChampions,playerScore9,item0,totalHealsOnTeammates,turretKills,bountyLevel,nexusKills,totalMinionsKilled,turretTakedowns,playerScore2,damagePerMinute,effectiveHealAndShielding,skillshotsDodged,visionClearedPings,mejaisFullStackInTime,playerAugment1,damageTakenOnTeamPercentage,playerScore1,playerAugment4,totalUnitsHealed,enemyJungleMonsterKills,wardsGuarded,playerSubteamId,visionScorePerMinute,playerScore7,item5,totalDamageShieldedOnTeammates,survivedThreeImmobilizesInFight,inhibitorKills,playerScore11,damageDealtToTurrets,tripleKills,killsWithHelpFromEpicMonster,damageDealtToBuildings,dodgeSkillShotsSmallWindow,summoner1Id,needVisionPings,goldPerMinute,riftHeraldTakedowns,teamElderDragonKills,takedownsFirstXMinutes,doubleAces,playerScore5,scuttleCrabKills,multiKillOneSpell,assistMePings,holdPings,summoner1Casts,baitPings,objectivesStolenAssists,playerScore6,totalAllyJungleMinionsKilled,soloKills,outnumberedKills,visionScore,item2,statPerks.flex,hadOpenNexus,statPerks.offense,largestCriticalStrike,turretPlatesTaken,playerScore0,summonerLevel,summoner2Casts,flawlessAces,playerScore3,killsUnderOwnTurret,Unnamed: 0,totalDamageDealt,physicalDamageTaken,enemyChampionImmobilizations,multikills,killsOnOtherLanesEarlyJungleAsLaner,subteamPlacement,dancedWithRiftHerald,physicalDamageDealt,getBackPings,spell4Casts,inhibitorsLost,twoWardsOneSweeperCount,playerScore9,detectorWardsPlaced,visionWardsBoughtInGame,acesBefore15Minutes,skillshotsHit,championTransform,totalEnemyJungleMinionsKilled,playerScore11,fullTeamTakedown,playerScore2,wardTakedownsBefore20M,damageDealtToObjectives,spell1Casts,killingSprees,legendaryCount,knockEnemyIntoTeamAndKill,junglerTakedownsNearDamagedEpicMonster,consumablesPurchased,wardsKilled,multiTurretRiftHeraldCount,outerTurretExecutesBefore10Minutes,gameLength,playerAugment3,item4,epicMonsterSteals,assists,magicDamageDealt,saveAllyFromDeath,epicMonsterStolenWithoutSmite,pickKillWithAlly,unseenRecalls,trueDamageTaken,poroExplosions,killsNearEnemyTurret,commandPings,12AssistStreakCount,controlWardsPlaced,participantId,placement,teamId,teamPosition,goldEarned,quickCleanse,basicPings,kTurretsDestroyedBeforePlatesFall,takedownsAfterGainingLevelAdvantage,initialBuffCount,teamRiftHeraldKills,baronTakedowns,kda,turretsTakenWithRiftHerald,championId,goldSpent,epicMonsterKillsNearEnemyJungler,turretsLost,enemyVisionPings,itemsPurchased,outnumberedNexusKill,perfectGame,quickFirstTurret,onMyWayPings,item6,takedownsBeforeJungleMinionSpawn,totalTimeSpentDead,profileIcon,deaths,deathsByEnemyChamps,nexusLost,playerScore7,playerScore5,buffsStolen,damageSelfMitigated,timePlayed,perfectDragonSoulsTaken,neutralMinionsKilled,quadraKills,trueDamageDealtToChampions,alliedJungleMonsterKills,takedownsInEnemyFountain,playerScore10,dragonTakedowns,elderDragonKillsWithOpposingSoul,playerAugment2,playerScore0,item3,totalHeal,champLevel,takedownsInAlcove,playerScore10,timeCCingOthers,killsOnRecentlyHealedByAramPack,moreEnemyJungleThanOpponent,largestMultiKill,totalDamageTaken,longestTimeSpentLiving,takedowns,tookLargeDamageSurvived,dragonKills,dang

In [110]:
df_joined.drop(['quickSoloKills', 'survivedSingleDigitHpCount', 'soloBaronKills', 'pushPings','playerScore3','playerScore8', 
                'getTakedownsInAllLanesEarlyJungleAsLaner', 'twentyMinionsIn3SecondsCount', 'playerScore6', 'summoner2Id', 'playerScore1', 'playerScore4', 
                'playerScore9', 'playerScore2', 'mejaisFullStackInTime', 'playerAugment1', 'playerScore1', 'playerAugment4', 'playerSubteamId', 
                'playerScore7','playerScore11', 'summoner1Id', 'playerScore5', 'puuid', 'playerScore6', 'playerScore0', 'summonerLevel', 'playerScore3', 
                'Unnamed: 0', 'subteamPlacement', 'playerScore9', 'championTransform', 'playerScore11', 'playerScore2', 'playerAugment3', 'poroExplosions', 
                'participantId', 'placement', 'teamId', 'quickCleanse', 'profileIcon', 'nexusLost', 'playerScore7', 'playerScore5', 'perfectDragonSoulsTaken', 
                'playerScore10', 'playerAugment2', 'playerScore0', 'playerScore4', 'largestKillingSpree'], axis=1, inplace=True)

In [113]:
df_joined.shape

(20, 208)